In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql
%sql sqlite:///store.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Задание 1
Вывести 5 самых больших по размеру (в байтах) треков жанра Blues и формата MPEG. Только название и размер.


In [17]:
%%sql
SELECT t.Name, t.Bytes
FROM tracks t
JOIN genres g ON t.GenreId = g.GenreId
JOIN media_types mt ON t.MediaTypeId
WHERE g.Name = 'Blues' and mt.Name = 'MPEG audio file'
ORDER BY t.Bytes DESC
LIMIT 5

 * sqlite:///store.db
Done.


Name,Bytes
Talkin' 'Bout Women Obviously,19161377
Riviera Paradise,17232776
Title Song,16501316
Old Love,15780747
Wiser Time,15161907


## Задание 2
Вывести названия всех треков и их исполнителей для всех треков жанра Rock, приобретенных сотрудниками Oracle.


In [20]:
%%sql
SELECT t.NAME, ar.Name
FROM invoice_items ii
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
JOIN customers c ON i.CustomerId = c.CustomerId
JOIN tracks t ON ii.TrackId = t.TrackId
JOIN genres g ON t.GenreId = g.GenreId
JOIN albums al ON t.AlbumId = al.AlbumId
JOIN artists ar ON al.ArtistId = ar.ArtistId
WHERE c.Company = 'Oracle' and g.Name = 'Rock'

 * sqlite:///store.db
Done.


Name,Name_1
Sunday Bloody Sunday,U2
New Year's Day,U2
That's The Way,Led Zeppelin
Ten Years Gone,Led Zeppelin
Achilles Last Stand,Led Zeppelin
Tea For One,Led Zeppelin
No Quarter,Led Zeppelin
Bad Moon Rising,Creedence Clearwater Revival
Down On The Corner,Creedence Clearwater Revival
Up Around The Bend,Creedence Clearwater Revival


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций (если 0, можно не выводить).


In [26]:
%%sql
SELECT c.Company as Company, COUNT(ii.TrackId) as jazz_tracks_count
FROM invoice_items ii
JOIN invoices i on ii.InvoiceId = i.InvoiceId
JOIN customers c on i.CustomerId = c.CustomerId
JOIN tracks  t ON ii.TrackId = t.TrackId
JOIN genres g on t.GenreId = g.GenreId
WHERE g.name = 'Jazz' and c.Company IS NOT NULL
GROUP by c.Company

 * sqlite:///store.db
Done.


Company,jazz_tracks_count
Apple Inc.,2
BMW,1
Google Inc.,12
JetBrains s.r.o.,3
Microsoft Corporation,15
Oracle,3
SAP,1
Telus,2


## Задание 4
Вывести топ 3 плейлиста по продолжительности (продолжительность плейлиста = сумма длин треков в нём). Только название и продолжительность (в часах).


In [27]:
%%sql
SELECT p.Name AS playlist_name, SUM(t.Milliseconds) / 3600000 AS total_duration_hours
FROM playlists p
JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId
JOIN tracks t ON pt.TrackId = t.TrackId
GROUP BY p.PlaylistId
ORDER BY total_duration_hours DESC
LIMIT 3

 * sqlite:///store.db
Done.


playlist_name,total_duration_hours
Hidden Gems,138
From That TV Show,138
90’s Music,10


## Задание 5
Для каждого набора (жанр, тип медиа) вывести количество треков в них, причем вывести только те наборы, для которых **все** треки стоят больше 1$ и для которых в наборе есть хотя бы один трек.

In [28]:
%%sql
SELECT g.Name AS genre_name, mt.Name as media_type_name, COUNT(t.TrackId) AS tracks_count
FROM tracks t
JOIN genres g ON t.GenreId = g.GenreId
JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId
Where t.UnitPrice > 1 AND NOT EXISTS (
  SELECT 1
  FROM tracks t_sub
  WHERE t_sub.GenreId = t.GenreId
  AND t_sub.MediaTypeId = t.MediaTypeId
  AND t_sub.UnitPrice <= 1
)
GROUP BY g.GenreId, g.Name, mt.MediaTypeId, mt.Name

 * sqlite:///store.db
Done.


genre_name,media_type_name,tracks_count
Blues,AAC audio file,9
Science Fiction,MPEG audio file,13
TV Shows,MPEG audio file,93
Sci Fi & Fantasy,MPEG audio file,26
Comedy,MPEG audio file,17
